# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

In [ ]:
pprint(establishments.find_one())

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': {'$eq': 20}}

# Use count_documents to display the number of documents in the result
print("Number of document in result:", establishments.count_documents(query))

# Display the first document in the results using pprint
results = establishments.find(query)
pprint(results[0])

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("Number of rows in DF:", establishments.count_documents(query))

# Display the first 10 rows of the DataFrame

df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex': 'London'}, 'RatingValue': {'$gte': '4'}}

# Use count_documents to display the number of documents in the result
print("Number of documents in result:", establishments.count_documents(query))

# Display the first document in the results using pprint
results = establishments.find(query)
pprint(results[0])

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("Number of rows in DF:", establishments.count_documents(query))

# Display the first 10 rows of the DataFrame
df.head(10)


### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
longitude = 0.08384000
latitude = 51.49014200
penang_flavours = {'BusinessName': {'$eq':'Penang Flavours'}}
geocode = establishments.find_one(penang_flavours, {'geocode': 1})['geocode']

geocode = establishments.find_one(penang_flavours, {'geocode': 1})['geocode']


In [ ]:
# Convert result to Pandas DataFrame
results = establishments.find(query).sort('scores.Hygiene', 1).limit(limit)
df = pd.DataFrame(results)
df

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
matches = {'$match': {'scores.Hygiene': {'$eq': 0}}}

groups = {'$group': {'_id': {'LocalAuthorityName': '$LocalAuthorityName'},
                         'count': {'$sum': 1}}}

sort = {'$sort': {'count': -1}}

# Print the number of documents in the result
pipeline = [matches, groups, sort]
# Print the first 10 results
results = list(establishments.aggregate(pipeline))
pprint(results[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
aggregated_df = pd.json_normalize(results)

aggregated_df.head()

In [ ]:
# Rename the columns
aggregated_df = aggregated_df.rename(columns={"count": "count",
                                              "_id.LocalAuthorityName": "_id"})
aggregated_df.head()

In [ ]:
# Reorder the columns
aggregated_df = aggregated_df[["_id", "count"]]

aggregated_df.head()

In [ ]:
# Display the number of rows in the DataFrame
print("Number of rows in DF:", establishments.count_documents(query))

# Display the first 10 rows of the DataFrame
aggregated_df.head(10)